# 🎮 Fantasy Football Projections - Control Panel

**Scrape and manage player projections from multiple sources.**

## Quick Start:
1. **Update `WEEK` and `SEASON`** in the configuration
2. **Run All Cells** to scrape all sources
3. View database status and data quality metrics

## Sources:
- 🔵 FanDuel
- 💤 Sleeper  
- 🔴 ESPN
- 📊 FantasyPros
- 🎯 First Down Studio

---


## 📦 Setup & Configuration


In [1]:
import os
import sqlite3
import subprocess
import sys
from datetime import datetime
from pathlib import Path

# Add scrapers directory to path FIRST (so local imports work)
NOTEBOOK_DIR = Path().absolute()
BACKEND_DIR = NOTEBOOK_DIR.parent
SCRAPERS_DIR = BACKEND_DIR / "scrapers"
sys.path.insert(0, str(SCRAPERS_DIR))

# Now import from scrapers (they use local imports like "from database import ProjectionsDB")
from database import ProjectionsDB
from scraper_sleeper import SleeperScraper
from scraper_espn import ESPNScraper
from scraper_fantasypros import FantasyProsScraper
from scraper_firstdown import FirstDownStudioScraper

# ==================== CONFIGURATION ====================
# Update these for each scraping session

WEEK = "Week 11"  # ⬅️ UPDATE THIS!
SEASON = "2025"
HEADLESS = True  # Set to False to see browser windows

# Database path
DB_PATH = str(BACKEND_DIR / "data" / "databases" / "projections.db")

print("✅ Setup Complete!")
print(f"📅 {WEEK}, Season {SEASON}")
print(f"👁️  Headless mode: {HEADLESS}")
print(f"💾 Database: {DB_PATH}")
print("\n" + "="*70 + "\n")


✅ Setup Complete!
📅 Week 11, Season 2025
👁️  Headless mode: True
💾 Database: C:\Users\Samer Faizi\Documents\Claude Model\backend\data\databases\projections.db




## 🤖 Run All Scrapers

This will scrape projections from all sources for the configured week.


In [2]:
def run_all_scrapers():
    """Run all scrapers for the configured week."""
    print(f"{'='*70}")
    print(f"RUNNING ALL SCRAPERS - {WEEK}")
    print(f"{'='*70}\n")
    
    results = {}
    
    # 1. Sleeper (API-based, fast)
    print("💤 Sleeper...")
    try:
        with SleeperScraper(db_path=DB_PATH) as scraper:
            scraper.scrape_and_save(week=WEEK, season=SEASON)
        results['Sleeper'] = '✅'
        print("  ✓ Complete\n")
    except Exception as e:
        results['Sleeper'] = f'❌ {str(e)[:50]}'
        print(f"  ✗ Error: {e}\n")
    
    # 2. ESPN
    print("🔴 ESPN...")
    try:
        with ESPNScraper(headless=HEADLESS, db_path=DB_PATH) as scraper:
            scraper.scrape_and_save(week=WEEK, season=SEASON)
        results['ESPN'] = '✅'
        print("  ✓ Complete\n")
    except Exception as e:
        results['ESPN'] = f'❌ {str(e)[:50]}'
        print(f"  ✗ Error: {e}\n")
    
    # 3. FantasyPros
    print("📊 FantasyPros...")
    try:
        with FantasyProsScraper(headless=HEADLESS, db_path=DB_PATH) as scraper:
            scraper.scrape_and_save(week=WEEK)
        results['FantasyPros'] = '✅'
        print("  ✓ Complete\n")
    except Exception as e:
        results['FantasyPros'] = f'❌ {str(e)[:50]}'
        print(f"  ✗ Error: {e}\n")
    
    # 4. First Down Studio
    print("🎯 First Down Studio...")
    try:
        with FirstDownStudioScraper(headless=HEADLESS, db_path=DB_PATH) as scraper:
            scraper.scrape_and_save(week=WEEK, scoring="PPR")
        results['First Down'] = '✅'
        print("  ✓ Complete\n")
    except Exception as e:
        results['First Down'] = f'❌ {str(e)[:50]}'
        print(f"  ✗ Error: {e}\n")
    
    # 5. FanDuel (via subprocess due to Playwright)
    print("🔵 FanDuel (subprocess)...")
    try:
        temp_script = NOTEBOOK_DIR / "_temp_fanduel.py"
        script_content = f'''
import sys
sys.path.insert(0, r"{SCRAPERS_DIR}")
from scraper_fanduel import FanDuelScraper

try:
    with FanDuelScraper(headless={HEADLESS}, db_path=r"{DB_PATH}") as scraper:
        scraper.scrape_and_save(week="{WEEK}")
    print("FanDuel complete")
except Exception as e:
    print(f"Error: {{e}}")
    raise
'''
        temp_script.write_text(script_content, encoding='utf-8')
        
        result = subprocess.run(
            [sys.executable, str(temp_script)],
            capture_output=True,
            text=True,
            cwd=str(SCRAPERS_DIR),
            timeout=300  # 5 minute timeout
        )
        
        if result.returncode == 0:
            results['FanDuel'] = '✅'
            print("  ✓ Complete\n")
        else:
            results['FanDuel'] = f'❌ {result.stderr[:50]}'
            print(f"  ✗ Error: {result.stderr}\n")
        
        temp_script.unlink(missing_ok=True)
    except Exception as e:
        results['FanDuel'] = f'❌ {str(e)[:50]}'
        print(f"  ✗ Error: {e}\n")
        if temp_script.exists():
            temp_script.unlink()
    
    # Summary
    print(f"{'='*70}")
    print(f"SCRAPING SUMMARY")
    print(f"{'='*70}\n")
    for source, status in results.items():
        print(f"  {source:<15} {status}")
    print(f"\n{'='*70}\n")

# Run scrapers
run_all_scrapers()


RUNNING ALL SCRAPERS - Week 11

💤 Sleeper...

Fetching Sleeper projections for 2025 Week 11...
Fetching all players from Sleeper API...
  ✓ Loaded 11401 players
Fetching from: https://api.sleeper.app/v1/projections/nfl/regular/2025/11
  ✓ Successfully fetched 8621 player projections

Parsing 8621 projections...
  Brandon Aubrey (K): 10.9 pts
  CeeDee Lamb (WR): 20.7 pts
  Oronde Gadsden (TE): 11.3 pts
  Kyle Pitts (TE): 10.3 pts
  Joe Flacco (QB): 17.7 pts
  Dak Prescott (QB): 22.1 pts
  Matthew Stafford (QB): 19.3 pts
  Khalil Shakir (WR): 13.4 pts
  James Cook (RB): 15.1 pts
  Jauan Jennings (WR): 10.7 pts
  Rachaad White (RB): 14.9 pts
  Sam Darnold (QB): 19.1 pts
  Kimani Vidal (RB): 15.3 pts
  Jaxon Smith-Njigba (WR): 22.0 pts
  Jaylen Waddle (WR): 15.6 pts
  TreVeyon Henderson (RB): 16.3 pts
  D'Andre Swift (RB): 12.8 pts
  Geno Smith (QB): 21.5 pts
  Tua Tagovailoa (QB): 19.6 pts
  Jalen Hurts (QB): 24.1 pts
  Ladd McConkey (WR): 14.5 pts
  Rome Odunze (WR): 12.8 pts
  Jordan Lo

## 📊 Database Status & Data Quality


In [3]:
def show_database_status():
    """Display comprehensive database status and data quality metrics."""
    with ProjectionsDB(db_path=DB_PATH) as db:
        all_projs = db.get_projections()
        
        print(f"{'='*70}")
        print(f"DATABASE STATUS")
        print(f"{'='*70}\n")
        print(f"Total records: {len(all_projs)}\n")
        
        # Count by source
        print(f"📊 Records by Source:")
        print("-" * 70)
        sources = {}
        for p in all_projs:
            source = p['source_website']
            sources[source] = sources.get(source, 0) + 1
        
        for source, count in sorted(sources.items()):
            print(f"  {source:<25} {count:>5} records")
        
        # Count by week
        print(f"\n📅 Records by Week:")
        print("-" * 70)
        weeks = {}
        for p in all_projs:
            week = p['week']
            weeks[week] = weeks.get(week, 0) + 1
        
        for week, count in sorted(weeks.items()):
            print(f"  {week:<25} {count:>5} records")
        
        # Team coverage (data quality check)
        print(f"\n🏈 Team Data Coverage (Quality Check):")
        print("-" * 70)
        for source in sorted(sources.keys()):
            source_projs = [p for p in all_projs if p['source_website'] == source]
            with_team = sum(1 for p in source_projs if p.get('team'))
            total = len(source_projs)
            pct = (with_team / total * 100) if total > 0 else 0
            status = "✅" if pct >= 95 else "⚠️" if pct >= 75 else "❌"
            print(f"  {source:<25} {with_team:>4}/{total:<4} ({pct:>5.1f}%) {status}")
        
        print(f"\n{'='*70}\n")

# Show status
show_database_status()


DATABASE STATUS

Total records: 3331

📊 Records by Source:
----------------------------------------------------------------------
  espn.com                    472 records
  fanduel.com                 981 records
  fantasypros.com             709 records
  firstdown.studio            337 records
  sleeper.com                 832 records

📅 Records by Week:
----------------------------------------------------------------------
  Week 10                    1625 records
  Week 11                    1706 records

🏈 Team Data Coverage (Quality Check):
----------------------------------------------------------------------
  espn.com                   472/472  (100.0%) ✅
  fanduel.com                981/981  (100.0%) ✅
  fantasypros.com            709/709  (100.0%) ✅
  firstdown.studio           337/337  (100.0%) ✅
  sleeper.com                832/832  (100.0%) ✅




---

## 🛠️ Utility Functions

**Use these as needed for database management**


In [6]:
# ============================================================================
# UTILITY FUNCTIONS - Uncomment to use
# ============================================================================

def clear_week(week_name):
    """Delete all records from a specific week."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM projections WHERE week = ?", (week_name,))
    deleted = cursor.rowcount
    conn.commit()
    conn.close()
    print(f"✓ Deleted {deleted} records from {week_name}")

def clear_source(source_name, week=None):
    """Delete records from a specific source."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    if week:
        cursor.execute("DELETE FROM projections WHERE source_website = ? AND week = ?", (source_name, week))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} for {week}")
    else:
        cursor.execute("DELETE FROM projections WHERE source_website = ?", (source_name,))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} (all weeks)")
    conn.commit()
    conn.close()

def clear_entire_database():
    """⚠️ DELETE ALL RECORDS - requires confirmation."""
    confirm = input("⚠️  Type 'DELETE ALL' to confirm: ")
    if confirm == "DELETE ALL":
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("DELETE FROM projections")
        conn.commit()
        deleted = cursor.rowcount
        conn.close()
        print(f"\n✓ Deleted {deleted} records from database")
    else:
        print("\n✗ Cancelled")

def show_week_source_matrix():
    """Display a matrix showing record counts by week and source."""
    import pandas as pd
    conn = sqlite3.connect(DB_PATH)
    query = """
        SELECT week, source_website, COUNT(*) as count
        FROM projections
        GROUP BY week, source_website
        ORDER BY week, source_website
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    if len(df) == 0:
        print("No data in database")
        return
    
    pivot = df.pivot(index='source_website', columns='week', values='count')
    pivot = pivot.fillna(0).astype(int)
    
    print(f"\n{'='*70}")
    print("WEEK × SOURCE MATRIX")
    print("="*70)
    print(pivot.to_string())
    print("="*70 + "\n")

def run_single_scraper(scraper_name):
    """Run a specific scraper by name."""
    scrapers = {
        'sleeper': lambda: SleeperScraper(db_path=DB_PATH),
        'espn': lambda: ESPNScraper(headless=HEADLESS, db_path=DB_PATH),
        'fantasypros': lambda: FantasyProsScraper(headless=HEADLESS, db_path=DB_PATH),
        'firstdown': lambda: FirstDownStudioScraper(headless=HEADLESS, db_path=DB_PATH)
    }
    
    name_lower = scraper_name.lower()
    if name_lower not in scrapers:
        print(f"Unknown scraper: {scraper_name}")
        print(f"Available: {', '.join(scrapers.keys())}")
        return
    
    print(f"Running {scraper_name}...")
    try:
        with scrapers[name_lower]() as scraper:
            if name_lower == 'sleeper':
                scraper.scrape_and_save(week=WEEK, season=SEASON)
            elif name_lower == 'espn':
                scraper.scrape_and_save(week=WEEK, season=SEASON)
            elif name_lower == 'firstdown':
                scraper.scrape_and_save(week=WEEK, scoring="PPR")
            else:
                scraper.scrape_and_save(week=WEEK)
        print(f"✓ {scraper_name} complete")
    except Exception as e:
        print(f"✗ Error: {e}")

# Uncomment to use:
# clear_week("Week 10")
# clear_source("fantasypros.com", WEEK)
# clear_entire_database()
show_week_source_matrix()
# run_single_scraper("sleeper")

print("✅ Utility functions loaded. Uncomment above to use.")



WEEK × SOURCE MATRIX
week              Week 10  Week 11
source_website                    
espn.com              232      240
fanduel.com           473      508
fantasypros.com       388      321
firstdown.studio      161      176
sleeper.com           371      461

✅ Utility functions loaded. Uncomment above to use.


---

## 📚 Quick Reference

### Weekly Workflow:
1. Update `WEEK` and `SEASON` in cell 2
2. **Kernel → Restart & Run All**
3. Wait for all scrapers to complete (~5-10 minutes)
4. Check database status for data quality

### Available Functions:
- `run_all_scrapers()` - Scrape all sources *(runs automatically)*
- `show_database_status()` - View database contents *(runs automatically)*
- `run_single_scraper(name)` - Run one scraper
- `clear_week(week)` - Delete all data for a week
- `clear_source(source, week)` - Delete data from a source
- `show_week_source_matrix()` - See week×source grid
- `clear_entire_database()` - Delete everything ⚠️

### Source Names:
- `"fanduel.com"`
- `"sleeper.com"`
- `"espn.com"`
- `"fantasypros.com"`
- `"firstdown.studio"`

### Tips:
- FanDuel uses Playwright and runs in a subprocess (this is normal)
- Set `HEADLESS = False` to see browser windows for debugging
- Scrapers typically take 1-3 minutes each
- Check "Team Data Coverage" to ensure quality (should be ≥95%)

---

**✅ Ready to use! Update WEEK and run all cells.**
